In [2]:
%load_ext kedro.ipython

[05/09/24 15:18:52] INFO     Registered line magic '%reload_kedro'                                   ]8;id=360830;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=31217;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#51\51]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=780699;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=329483;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#53\53]8;;\

                    INFO     Resolved project path as: /Users/gavinlou/Developer/gasai.             ]8;id=972994;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=48451;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/09/24 15:18:56] INFO     Kedro project gasAI                                                    ]8;id=518418;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=964871;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=398127;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=895035;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

[05/09/24 15:18:57] INFO     Registered line magic 'run_viz'                                        ]8;id=880650;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=652498;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [ ]:
%reload_ext kedro.ipython

In [3]:
data_set = catalog.load("s3_conc_aligned_df")

[05/09/24 15:20:14] INFO     Loading data from s3_conc_aligned_df (ParquetDataset)...           ]8;id=414838;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=91826;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

In [15]:
data_set.head()

,exp_no,timestamp_bin,A1_Resistance,A1_Resistance_diff,A1_Resistance_norm,A1_Sensor,A1_Sensor_diff,A1_Sensor_norm,SHT40_Humidity,SHT40_temp,index,resistance_ratio,ace_conc,expo_time
0,0,0.0,720650.750,0.000,1.000000,4518.0,0.0,1.000000,42.835,29.430,4481.5,1.768473,3.033000e-07,3.0
1,0,1.0,720361.875,0.000,0.999599,4519.5,0.0,1.000332,42.840,29.435,4483.5,1.768473,3.033000e-07,3.0
2,0,2.0,719688.470,-673.405,0.998665,4523.0,3.5,1.001107,42.845,29.445,4485.5,1.768473,3.033000e-07,3.0
3,0,3.0,719015.940,-1634.810,0.997731,4526.5,8.5,1.001881,42.830,29.435,4487.5,1.768473,3.033000e-07,3.0
4,0,4.0,718727.905,-1345.095,0.997332,4528.0,7.0,1.002213,42.845,29.440,4489.5,1.768473,3.033000e-07,3.0


In [35]:
df = data_set[["exp_no","timestamp_bin","A1_Resistance","resistance_ratio"]]
df[df["exp_no"]==1].tail()

,exp_no,timestamp_bin,A1_Resistance,resistance_ratio
3642,1,1825.0,725202.970,1.816453
3643,1,1826.0,725690.250,1.816453
3644,1,1827.0,726275.345,1.816453
3645,1,1828.0,726275.345,1.816453
3646,1,1829.0,726470.785,1.816453


In [22]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

In [36]:
features = torch.tensor(df[['timestamp_bin', 'A1_Resistance']].values).float()
targets = torch.tensor(df['resistance_ratio'].values).float().unsqueeze(1)  # Ensure target is 2D
groups = torch.tensor(df['exp_no'].values)

# Group-wise shuffle and split
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, val_idx = next(gss.split(features.numpy(), groups=groups.numpy()))

# Normalize features
scaler = StandardScaler()
train_features = features[train_idx]
scaler.fit(train_features.numpy())  # Fit only on training data

scaled_train_features = torch.tensor(scaler.transform(train_features.numpy()))
scaled_val_features = torch.tensor(scaler.transform(features[val_idx].numpy()))

# Custom dataset
class ResistanceDataset(Dataset):
    def __init__(self, features, targets):
        # Ensure features are in the shape [batch_size, num_channels, sequence_length]
        self.features = features.unsqueeze(2)  # Add a new dimension for sequence_length
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.targets[index]

# Update datasets with scaled features
train_dataset = ResistanceDataset(scaled_train_features, targets[train_idx])
val_dataset = ResistanceDataset(scaled_val_features, targets[val_idx])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 16)  # Adjust the input features depending on the output size after conv and pool layers
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [47]:
from torch.optim import Adam

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        
        print(f'Epoch {epoch+1} Train Loss: {epoch_loss:.4f} Val Loss: {val_loss:.4f}')

# Training the model
train_model(model, train_loader, val_loader, n_epochs=50)


Epoch 1 Train Loss: 0.0052 Val Loss: 0.0021
Epoch 2 Train Loss: 0.0021 Val Loss: 0.0019
Epoch 3 Train Loss: 0.0020 Val Loss: 0.0015
Epoch 4 Train Loss: 0.0019 Val Loss: 0.0015
Epoch 5 Train Loss: 0.0019 Val Loss: 0.0018
Epoch 6 Train Loss: 0.0018 Val Loss: 0.0015
Epoch 7 Train Loss: 0.0018 Val Loss: 0.0015
Epoch 8 Train Loss: 0.0018 Val Loss: 0.0016
Epoch 9 Train Loss: 0.0018 Val Loss: 0.0014
Epoch 10 Train Loss: 0.0017 Val Loss: 0.0016
Epoch 11 Train Loss: 0.0017 Val Loss: 0.0014
Epoch 12 Train Loss: 0.0017 Val Loss: 0.0015
Epoch 13 Train Loss: 0.0017 Val Loss: 0.0015
Epoch 14 Train Loss: 0.0017 Val Loss: 0.0014
Epoch 15 Train Loss: 0.0017 Val Loss: 0.0018


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│   34 │   │   print(f'Epoch {epoch+1} Train Loss: {epoch_loss:.4f} Val Loss: {val_loss:.4f}')     │
│   35                                                                                             │
│   36 # Training the model                                                                        │
│ ❱ 37 train_model(model, train_loader, val_loader, n_epochs=50)                                   │
│   38                                                                                             │
│                                                                                                  │
│ in train_model:16                                                                                │
│                                                                                                  │
│   13 │   │   for inputs, labels in train_loader:                                                 │
│   14 │   │   │   inputs, labels = inputs.to(device), labels.to(device)                           │
│   15 │   │   │   optimizer.zero_grad()                                                           │
│ ❱ 16 │   │   │   outputs = model(inputs)                                                         │
│   17 │   │   │   loss = criterion(outputs, labels)                                               │
│   18 │   │   │   loss.backward()                                                                 │
│   19 │   │   │   optimizer.step()                                                                │
│                                                                                                  │
│ /Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/torch/nn/modules │
│ /module.py:1532 in _wrapped_call_impl                                                            │
│                                                                                                  │
│   1529 │   │   if self._compiled_call_impl is not None:                                          │
│   1530 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1531 │   │   else:                                                                             │
│ ❱ 1532 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1533 │                                                                                         │
│   1534 │   def _call_impl(self, *args, **kwargs):                                                │
│   1535 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ /Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/torch/nn/modules │
│ /module.py:1541 in _call_impl                                                                    │
│                                                                                                  │
│   1538 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1539 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1540 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1541 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1542 │   │                                                                                     │
│   1543 │   │   try:                                                                              │
│   1544 │   │   │   result = None                           

In [29]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1: Train Loss: 0.0032, Val RMSE: 0.0400
Epoch 2: Train Loss: 0.0019, Val RMSE: 0.0365
Epoch 3: Train Loss: 0.0018, Val RMSE: 0.0406
Epoch 4: Train Loss: 0.0018, Val RMSE: 0.0381
Epoch 5: Train Loss: 0.0018, Val RMSE: 0.0394
Epoch 6: Train Loss: 0.0017, Val RMSE: 0.0438
Epoch 7: Train Loss: 0.0017, Val RMSE: 0.0400
Epoch 8: Train Loss: 0.0017, Val RMSE: 0.0373
Epoch 9: Train Loss: 0.0017, Val RMSE: 0.0445
Epoch 10: Train Loss: 0.0017, Val RMSE: 0.0439
